# Lecture 2: Advanced Python
## ECON5170 Computational Methods in Economics


## Speed

* Efficient computation in R.

* R is a vector-oriented language. 
  * In most cases, vectorization speeds up computation.
* Multiple CPUs for parallel execution
  * Save time after optimizing the code for speed.


## Vectorization

* Mathematical equivalence $\neq$ computation equivalence

* Speed matter in
  * Structural estimation
  * Big data
  * Simulations
  * Hyper parameter tuning



* For example, @lin2020's preferred algorithm 
  * 8 hours on a 24-core = 192 core hours

* Code optimization is essential for such problems.

* Optimizing code takes human time.

* Tradeoff between human time and computer time.

### Econometrics Example

In OLS regression, under heteroskedasticity
$
\sqrt{n}\left(\widehat{\beta}-\beta_{0}\right)\stackrel{d}{\to}N\left(0,E\left[x_{i}x_{i}'\right]^{-1}\mathrm{var}\left(x_{i}e_{i}\right)E\left[x_{i}x_{i}'\right]^{-1}\right)
$
where $\mathrm{var}\left(x_{i}e_{i}\right)$ can be estimated by 

$$
\underset{\mathrm{opt1}}{\frac{1}{n}\sum_{i=1}^{n}x_{i}x_{i}'\widehat{e}_{i}^{2}}=\underset{\mathrm{opt2,3}}{\frac{1}{n}X'DX}=\underset{\mathrm{opt 4}}{\frac{1}{n}\left(X'D^{1/2}\right)\left(D^{1/2}X\right)}
$$

where $D$ is a diagonal matrix of $\left(\widehat{e}_{1}^{2},\widehat{e}_{2,}^{2},\ldots,\widehat{e}_{n}^{2}\right)$.

At least 4 mathematically equivalent ways:

1. literally sum $\hat{e}_i^2 x_i x_i'$  over $i=1,\ldots,n$ one by one.
2. $X' \mathrm{diag}(\hat{e}^2) X$, with a dense central matrix.
3. $X' \mathrm{diag}(\hat{e}^2) X$, with a sparse central matrix.
4. Do cross product to `X*e_hat`. It takes advantage of the element-by-element operation in R.


In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix 
import random
import datetime
import math
import statistics
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-4yjgc0zc because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def lpm(n):
    # estimation in a linear probability model

    # set the parameters
    b0 = np.array([-1, 1])

    # generate the data
    e = np.random.normal(size=n)
    X = np.hstack((np.ones((n, 1)), np.random.normal(size=(n, 1))))
    Y = (X @ b0 + e >= 0)

    # OLS estimation
    bhat = np.linalg.inv(X.T @ X) @ X.T @ Y

    # calculate the t-value
    bhat2 = bhat[1]  # parameter we want to test
    e_hat = Y - X @ bhat
    return X, e_hat


We run the 4 estimators for the same data, and compare the time.

In [3]:
import time
import numpy as np
from scipy.sparse import diags

# an example of robust variance matrix.
# compare the implementation via matrix and vectorization.

# n = 5000; Rep = 10 # large matrix

n = 50; Rep = 1000 # small matrix



for opt in range(1, 5):
    pts0 = time.time()
    
    # initialize the matrix for computing the variance-covariance matrix
    XXe2 = np.zeros((2, 2))

    # loop over the replications and compute the variance-covariance matrix
    for iter in range(Rep):
        np.random.seed(iter)
        data = lpm(n)
        X = data[0]
        e_hat = data[1]
        # compute the variance-covariance matrix using element-by-element calculation
        if opt == 1:
            for i in range(n):
                XXe2 += e_hat[i]**2 * np.matrix(X[i,]).T @ np.matrix(X[i,])
        
        # compute the variance-covariance matrix using matrix multiplication with dense matrices
        elif opt == 2:
            e_hat2_M = np.zeros((n, n))
            np.fill_diagonal(e_hat2_M, e_hat**2)
            XXe2 = np.matrix(X).T @ np.matrix(e_hat2_M) @ np.matrix(X)
        
        # compute the variance-covariance matrix using matrix multiplication with sparse matrices
        elif opt == 3:
            e_hat2_M = diags(e_hat**2, format='csr')
            XXe2 = X.T @ e_hat2_M @ X
        
        # compute the variance-covariance matrix using vectorization with no waste
        elif opt == 4:
            e_hat = e_hat.reshape((-1, 1))
            Xe = np.matrix(X).T * np.matrix(e_hat)
            XXe2 = Xe @ Xe.T
        
        # compute the robust variance-covariance matrix using the sample size
        XX_inv = np.linalg.inv(X.T @ X)
        sig_B = XX_inv @ XXe2 @ XX_inv
        
    print("n =", n, ", Rep =", Rep, ", opt =", opt, ", time =", time.time() - pts0, "\n")


n = 50 , Rep = 1000 , opt = 1 , time = 0.9605729579925537 

n = 50 , Rep = 1000 , opt = 2 , time = 0.10442757606506348 

n = 50 , Rep = 1000 , opt = 3 , time = 0.4029197692871094 

n = 50 , Rep = 1000 , opt = 4 , time = 0.15856575965881348 



* When $n$ is small
  * `matrix` is fast
  * `Matrix` is slow
  * Vectorized version is the fastest.

* When $n$ is big
  * `matrix` is slow
  * `Matrix` is fast
  * Vectorized version is still the fastest.

### Real-data example 

* `data_example/IPUMS.R` with 234 thousand observations
*  Vectorization can significantly saves computing time in more complicated operations

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Read CSV files
d99 = pd.read_csv("data_example/ipums.la.99.csv", header=None)
d98 = pd.read_csv("data_example/ipums.la.98.csv", header=None)
d97 = pd.read_csv("data_example/ipums.la.97.csv", header=None)

# Concatenate dataframes
d0 = pd.concat([d97, d98, d99])

# Create X matrix
X = np.c_[np.ones(d0.shape[0]), d0[[20, 22]].values]

# Create y vector
y = d0[11].values

# Perform linear regression
reg = LinearRegression(fit_intercept=False)
reg.fit(X, y)

# Calculate predicted values
y_pred = reg.predict(X)

# Calculate residuals
e_hat = y - y_pred

# Get number of observations (n) and number of predictors (K)
n = d0.shape[0]
K = X.shape[1]



In [4]:
import numpy as np
import time
from scipy import sparse

options = [1, 3, 4]
K = X.shape[1]

for opt in options:
    pts0 = time.time()
    XXe2 = np.zeros((K, K))
    
    if opt == 1:
        for i in range(n):
            XXe2 += e_hat[i]**2 * np.outer(X[i, :], X[i, :])
    elif opt == 3:
        e_hat2_M = sparse.diags(e_hat**2, 0, format='csr')
        XXe2 = X.T @ e_hat2_M @ X
    elif opt == 4:
        Xe = X * e_hat[:, np.newaxis]
        XXe2 = Xe.T @ Xe
    
    print("outcome =", XXe2.flatten(), ", opt =", opt, ", time =", time.time() - pts0)


outcome = [  41002.57302057  186902.08983748    5879.07973157  186902.08983748
 1175914.30007041   35414.11305207    5879.07973157   35414.11305207
   10269.50322852] , opt = 1 , time = 1.2031960487365723
outcome = [  41002.57302058  186902.08983737    5879.07973157  186902.08983737
 1175914.30006964   35414.11305207    5879.07973157   35414.11305207
   10269.50322852] , opt = 3 , time = 0.011132478713989258
outcome = [  41002.57302058  186902.08983737    5879.07973157  186902.08983737
 1175914.30006964   35414.11305207    5879.07973157   35414.11305207
   10269.50322852] , opt = 4 , time = 0.00201416015625


## Efficient Loop

* R evolves for big data
* housekeeping is needed in `for` loops
* `plyr` simplifies the job and facilitates parallelization.


### Example

* Empirical coverage probability of a Poisson distribution
* Write a DIY `CI` for confidence interval

This is a standard `for` loop.

In [2]:
import numpy as np
import time

def CI(x):
    # x is a numpy array of random variables
    n = len(x)
    mu = np.mean(x)
    sig = np.std(x)
    upper = mu + 1.96 / np.sqrt(n) * sig
    lower = mu - 1.96 / np.sqrt(n) * sig
    return {"lower": lower, "upper": upper}

This is a standard `for` loop.

In [4]:
Rep = 100 # or whatever value you choose
sample_size = 100 # or whatever value you choose
mu = 10 # or whatever value you choose

np.random.seed(1) # set seed for reproducibility
out = [] # initialize out as empty list
pts0 = time.time() # check time

for i in range(Rep):
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    out.append((bounds["lower"] <= mu) & (mu <= bounds["upper"]))

pts1 = time.time() - pts0 # check time elapsed
print("loop without pre-definition takes", pts1, "seconds")

loop without pre-definition takes 0.008306026458740234 seconds


In [5]:
import numpy as np
import time


Rep = 100 # or whatever value you choose
sample_size = 10 # or whatever value you choose
mu = 10 # or whatever value you choose

np.random.seed(1) # set seed for reproducibility
out = np.zeros(Rep, dtype=bool) # pre-define out with appropriate shape
pts0 = time.time() # check time

for i in range(Rep):
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    out[i] = ((bounds["lower"] <= mu) & (mu <= bounds["upper"]))

pts1 = time.time() - pts0 # check time elapsed
print("loop with pre-definition takes", pts1, "seconds")

loop with pre-definition takes 0.008758306503295898 seconds


* Memoery operates differently with or without the container

## Parallel Computing

Parallel computing becomes essential when the data size is beyond the storage of a single computer, for example  {% cite li2018embracing %}.
Here we explore the speed gain of parallel computing on a multicore machine.

Here we explore the speed gain of parallel computing on a multicore machine.

The package `multiprocessing` is the choice for parallel computing in Python.
Below is the basic structure. 

### Example

* Two CPUs running simultaneously, in theory cut the time to a half of that on a single CPU

* Compare the speed of a parallel loop and a single-core sequential loop.

In [11]:
import time
from multiprocessing import Pool
import multiprocessing as mp

def parallel_func(i):
    # your function code here
    # for example:
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    return (bounds["lower"] <= mu) & (mu <= bounds["upper"])

Rep = 100 # or whatever value you choose
mu = 10 # or whatever value you choose
sample_size = 10 # or whatever value you choose

pool = Pool(2) # open 2 CPUs
pts0 = time.time() # check time

out = pool.map(parallel_func, range(Rep))

pts1 = time.time() - pts0 # check time elapsed
print("parallel loop takes", pts1, "seconds")


parallel loop takes 0.0038094520568847656 seconds


* Surprisingly, parallel computing runs more slowly
  * Each loop can be done in very short time.

In [12]:
import time
from multiprocessing import Pool

def parallel_func(i):
    # your function code here
    # for example:
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    return (bounds["lower"] <= mu) & (mu <= bounds["upper"])

Rep = 200 # or whatever value you choose
mu = 10 # or whatever value you choose
sample_size = 2000000 # or whatever value you choose

# parallel version with 2 cores
pool = Pool(2) # open 2 CPUs
pts0 = time.time() # check time
out = pool.map(parallel_func, range(Rep))
print("2-core parallel loop takes", time.time() - pts0 , "seconds")


2-core parallel loop takes 19.011606454849243 seconds


In [13]:

# single-core version
pts0 = time.time()
out = [parallel_func(i) for i in range(Rep)]
print("single-core loop takes", time.time() - pts0 , "seconds")

single-core loop takes 36.475889682769775 seconds


If we have two CPUs running simultaneously, in theory we can cut the time to a half of that on a single CPU. Is that what happening in practice?

### Multiprocessing with the `process` class

In [22]:
Rep = 10
sample_size = 1000
mu = 2

import multiprocessing as mp
from multiprocessing import Pool
from multiprocessing import Process


def capture(i,return_dict):
    np.random.seed(i)
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    result = ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    # process_id = os.getpid()
    # print("Process ID: " + str(process_id))
    print("The result: " + str(result))
    return_dict[i]=result

pts0 = time.time() # check time

manager = mp.Manager()
return_dict = manager.dict()
jobs = []

for i in range(Rep):
    p = Process(target=capture, args=(i,return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

print(return_dict.values())

stat_cover = np.mean(return_dict.values()) # jobs.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")

The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
[True, True, True, True, True, True, True, True, True, True]
empirical coverage probability =  1.0 % 

The the calculation time is: 0.1193692684173584 



### Multiprocessing with the `pool` class & `apply()`

In [26]:
Rep = 200
sample_size = 2000
mu = 2


pts0 = time.time()  # check time

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process wit apply()
results = [pool.apply(capture, args=(i,)) for x in range(Rep)]

print( "empirical coverage probability = ", np.mean(results), "\n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  0.955 

The the calculation time is: 0.17428827285766602 



### Multiprocessing with the `pool` class & `map()`

In [28]:
Rep = 200
sample_size = 2000
mu = 2

pts0 = time.time()  # check time
def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    
# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process with the map()
results = pool.map(capture, range(Rep), )

print( "empirical coverage probability = ", results.count(True)/Rep*100, "\n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  94.5 

The the calculation time is: 0.035188913345336914 



## Remote Computing

Investing money from our own pocket to an extremely powerful laptop to conduct heavy-lifting computational work
is unnecessary. (i) We do not run these long jobs every day, it is more cost efficient
to share a workhorse. (ii) We cannot keep our laptop always on when we move it
around. The right solution is remote computing on a server.



No fundamental difference lies between local and remote computing.
We prepare the data and code, open a shell for communication, run the code, and collect the results.
One potential obstacle is dealing with a command-line-based operation system.
Such command line tools is the norm of life two or three decades ago, but today we mostly
work in a graphic operating system like Windows or OSX.
For Windows users (I am one of them), I recommend [PuTTY](http://www.putty.org/), a shell, and [WinSCP](http://winscp.net/eng/download.php), a graphic interface for input and output.


Most servers in the world are running Unix/Linux operation system.
Here are a few commands for basic operations.

* mkdir
* cd
* copy
* top
* screen
* ssh user@address
* start a program


Our department's computation infrastructure has been improving.
A server dedicated to  professors is a 16-core machine. I have opened an account for you.
You can try out this script on `econsuper`.

1. Log in `econsuper.econ.cuhk.edu.hk`;
2. Save the code block below as `loop_server.R`, and upload it to the server;
3. In a shell, run `R --vanilla <loop_server.R> result_your_name.out`;
4. To run a command in the background, add `&` at the end of the above command. To keep it running after closing the console, add `nohup` at the beginning of the command.